In [1]:
import json
import os
import shutil
from pathlib import Path
import random
from glob import glob

# Đường dẫn tới các folders
base_path = Path("/home/hungnq/hungnq/sd_stuff/notebook/experiment_2")
folders = [
    "prompt_variations_images_batch_0_50_2",
    "prompt_variations_images_batch_50_100",
    "prompt_variations_images_batch_100_150",
    "prompt_variations_images_batch_150_200_2"
]

# Đọc tất cả metadata từ 4 folders
all_images = []  # Lưu trữ (folder, img_folder, image_path, metadata_key)
all_metadata = {}

for folder in folders:
    metadata_path = base_path / folder / "metadata.json"
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
        all_metadata.update(metadata)
        
        # Lấy danh sách image folders trong folder này
        folder_path = base_path / folder
        image_folders = [d for d in os.listdir(folder_path) if d.startswith('image_') and os.path.isdir(folder_path / d)]
        
        for img_folder in image_folders:
            # Tìm tất cả ảnh trong img_folder
            img_folder_path = folder_path / img_folder
            image_files = list(img_folder_path.glob('*.png')) + list(img_folder_path.glob('*.jpg'))
            
            for img_file in image_files:
                # Tìm metadata key tương ứng
                img_id = img_folder.split('_')[1]  # '0001'
                for key in all_metadata.keys():
                    if f"img{img_id}_" in key and img_file.stem in key:
                        all_images.append((folder, img_folder, img_file, key))
                        break

print(f"Tổng số ảnh: {len(all_images)}")
print(f"Tổng số metadata entries: {len(all_metadata)}")

# Chọn 200 ảnh ngẫu nhiên
random.seed(42)  # Để kết quả có thể tái tạo
selected_images = random.sample(all_images, min(200, len(all_images)))

print(f"Đã chọn {len(selected_images)} ảnh")

# Tạo folder selected_200_ids (xóa nếu đã tồn tại để tránh lỗi)
output_folder = base_path / "selected_200_ids"
if output_folder.exists():
    shutil.rmtree(output_folder)
output_folder.mkdir(exist_ok=True)

# Copy các ảnh đã chọn với tên đánh số từ 1 đến 200
selected_metadata = {}
for idx, (source_folder, img_folder, img_path, metadata_key) in enumerate(selected_images, start=1):
    # Lấy extension của ảnh
    ext = img_path.suffix
    
    # Tên file mới: 1.png, 2.png, ..., 200.png
    new_filename = f"{idx}{ext}"
    dest_path = output_folder / new_filename
    
    # Copy ảnh
    shutil.copy2(img_path, dest_path)
    
    # Cập nhật metadata
    if metadata_key in all_metadata:
        new_value = all_metadata[metadata_key].copy()
        new_value['local_path'] = f"selected_200_ids/{new_filename}"
        new_value['new_id'] = idx
        new_value['original_folder'] = source_folder
        new_value['original_image_folder'] = img_folder
        selected_metadata[f"img{idx:04d}"] = new_value

print(f"Đã copy {len(selected_images)} ảnh vào {output_folder}")

# Lưu metadata mới
metadata_output_path = output_folder / "metadata.json"
with open(metadata_output_path, 'w') as f:
    json.dump(selected_metadata, f, indent=2)

print(f"Đã lưu metadata với {len(selected_metadata)} entries vào {metadata_output_path}")
print(f"\nHoàn thành! Folder selected_200_ids đã được tạo tại: {output_folder}")
print(f"Tất cả ảnh được đánh số từ 1 đến {len(selected_images)}")

Tổng số ảnh: 164
Tổng số metadata entries: 152
Đã chọn 164 ảnh
Đã copy 164 ảnh vào /home/hungnq/hungnq/sd_stuff/notebook/experiment_2/selected_200_ids
Đã lưu metadata với 164 entries vào /home/hungnq/hungnq/sd_stuff/notebook/experiment_2/selected_200_ids/metadata.json

Hoàn thành! Folder selected_200_ids đã được tạo tại: /home/hungnq/hungnq/sd_stuff/notebook/experiment_2/selected_200_ids
Tất cả ảnh được đánh số từ 1 đến 164


In [3]:
# Điều tra chi tiết số lượng ảnh trong mỗi folder
import json
from pathlib import Path

base_path = Path("/home/hungnq/hungnq/sd_stuff/notebook/experiment_2")
folders = [
    "prompt_variations_images_batch_0_50",
    "prompt_variations_images_batch_50_100",
    "prompt_variations_images_batch_100_150",
    "prompt_variations_images_batch_150_200_2"
]

total_images = 0
folder_stats = {}

for folder in folders:
    folder_path = base_path / folder
    
    # Đếm số image folders
    image_folders = [d for d in os.listdir(folder_path) if d.startswith('image_') and os.path.isdir(folder_path / d)]
    
    # Đếm tổng số ảnh
    image_count = 0
    image_details = []
    
    for img_folder in image_folders:
        img_folder_path = folder_path / img_folder
        images = list(img_folder_path.glob('*.png')) + list(img_folder_path.glob('*.jpg'))
        image_count += len(images)
        image_details.append({
            'folder': img_folder,
            'num_images': len(images),
            'images': [img.name for img in images]
        })
    
    folder_stats[folder] = {
        'num_image_folders': len(image_folders),
        'total_images': image_count,
        'details': image_details
    }
    
    total_images += image_count
    
    print(f"\n{folder}:")
    print(f"  - Số image folders: {len(image_folders)}")
    print(f"  - Tổng số ảnh: {image_count}")
    
    # Kiểm tra nếu có folder nào có nhiều hơn 1 ảnh
    multi_image_folders = [d for d in image_details if d['num_images'] > 1]
    if multi_image_folders:
        print(f"  - Folders có nhiều hơn 1 ảnh: {len(multi_image_folders)}")
        for item in multi_image_folders[:5]:  # Show first 5
            print(f"    + {item['folder']}: {item['num_images']} ảnh - {item['images']}")
    
    # Kiểm tra nếu có folder nào không có ảnh
    empty_folders = [d for d in image_details if d['num_images'] == 0]
    if empty_folders:
        print(f"  - Folders không có ảnh: {len(empty_folders)}")
        for item in empty_folders[:5]:
            print(f"    + {item['folder']}")

print(f"\n{'='*60}")
print(f"TỔNG CỘNG: {total_images} ảnh từ tất cả các folders")
print(f"{'='*60}")


prompt_variations_images_batch_0_50:
  - Số image folders: 39
  - Tổng số ảnh: 51
  - Folders có nhiều hơn 1 ảnh: 4
    + image_0001: 5 ảnh - ['var2_male.png', 'var3_male.png', 'var4_male.png', 'var1_male.png', 'base_male.png']
    + image_0002: 4 ảnh - ['var2_male.png', 'var3_male.png', 'var4_male.png', 'var1_male.png']
    + image_0003: 4 ảnh - ['var2_male.png', 'var3_male.png', 'var1_male.png', 'base_male.png']
    + image_0005: 3 ảnh - ['var2_male.png', 'var1_male.png', 'base_male.png']

prompt_variations_images_batch_50_100:
  - Số image folders: 48
  - Tổng số ảnh: 48

prompt_variations_images_batch_100_150:
  - Số image folders: 37
  - Tổng số ảnh: 36
  - Folders không có ảnh: 1
    + image_0037

prompt_variations_images_batch_150_200_2:
  - Số image folders: 45
  - Tổng số ảnh: 42
  - Folders không có ảnh: 3
    + image_0040
    + image_0046
    + image_0017

TỔNG CỘNG: 177 ảnh từ tất cả các folders


In [4]:
# Phân tích file CSV để kiểm tra số lượng prompts
import pandas as pd
from pathlib import Path

base_path = Path("/home/hungnq/hungnq/sd_stuff/notebook/experiment_2")
csv_file = base_path / "prompt_base_and_variations_full.csv"

# Đọc CSV
df = pd.read_csv(csv_file)

print(f"Tổng số dòng trong CSV: {len(df)}")
print(f"Các cột: {df.columns.tolist()}")
print()

# Kiểm tra số lượng ảnh unique
unique_images = df['Images'].unique()
print(f"Số lượng ảnh unique: {len(unique_images)}")
print()

# Kiểm tra các cột prompt
prompt_columns = [col for col in df.columns if 'prompt' in col.lower()]
print(f"Các cột prompt: {prompt_columns}")
print()

# Đếm số dòng có base_prompt
has_base_prompt = df['base_prompt'].notna().sum()
print(f"Số dòng có base_prompt: {has_base_prompt}")

# Đếm số dòng có variation prompts
for col in prompt_columns:
    if 'variation' in col:
        has_prompt = df[col].notna().sum()
        print(f"Số dòng có {col}: {has_prompt}")

print("\n" + "="*60)

# Kiểm tra các ảnh trong CSV vs ảnh trong selected_200_ids
selected_folder = base_path / "selected_200_ids"
if selected_folder.exists():
    # Đọc metadata
    metadata_file = selected_folder / "metadata.json"
    with open(metadata_file, 'r') as f:
        metadata = json.load(f)
    
    print(f"\nSố ảnh trong selected_200_ids: {len(metadata)}")
    
    # Lấy danh sách image_name từ metadata
    image_names_in_selected = set()
    for key, value in metadata.items():
        if 'image_name' in value:
            image_names_in_selected.add(value['image_name'])
    
    print(f"Số image_name unique trong metadata: {len(image_names_in_selected)}")
    
    # So sánh với CSV
    image_names_in_csv = set(df['Images'].tolist())
    print(f"Số Images trong CSV: {len(image_names_in_csv)}")
    
    # Tìm ảnh có trong selected nhưng không có trong CSV
    missing_in_csv = image_names_in_selected - image_names_in_csv
    if missing_in_csv:
        print(f"\nẢnh có trong selected_200_ids nhưng KHÔNG có trong CSV: {len(missing_in_csv)}")
        print(f"Danh sách: {sorted(missing_in_csv)[:10]}")
    
    # Tìm ảnh có trong CSV nhưng không có trong selected
    missing_in_selected = image_names_in_csv - image_names_in_selected
    if missing_in_selected:
        print(f"\nẢnh có trong CSV nhưng KHÔNG có trong selected_200_ids: {len(missing_in_selected)}")
        print(f"Danh sách (10 đầu tiên): {sorted(missing_in_selected)[:10]}")
    
    # Tìm ảnh có ở cả hai
    in_both = image_names_in_selected & image_names_in_csv
    print(f"\nẢnh có ở CẢ HAI nơi: {len(in_both)}")
    
print("="*60)

Tổng số dòng trong CSV: 199
Các cột: ['Images', 'gender', 'image_path', 'base_prompt', 'variation_1_prompt', 'variation_2_prompt', 'variation_3_prompt', 'variation_4_prompt']

Số lượng ảnh unique: 146

Các cột prompt: ['base_prompt', 'variation_1_prompt', 'variation_2_prompt', 'variation_3_prompt', 'variation_4_prompt']

Số dòng có base_prompt: 199
Số dòng có variation_1_prompt: 4
Số dòng có variation_2_prompt: 4
Số dòng có variation_3_prompt: 3
Số dòng có variation_4_prompt: 2


Số ảnh trong selected_200_ids: 164
Số image_name unique trong metadata: 129
Số Images trong CSV: 146

Ảnh có trong CSV nhưng KHÔNG có trong selected_200_ids: 17
Danh sách (10 đầu tiên): ['13.jpg', '14.jpg', '24.jpg', '28.jpg', '3.jpg', '30.jpg', '4.jpg', '41.jpg', '42.jpg', 'Image_25.jpg']

Ảnh có ở CẢ HAI nơi: 129


In [5]:
# Tìm ảnh bị lặp trong CSV
import pandas as pd
from pathlib import Path

base_path = Path("/home/hungnq/hungnq/sd_stuff/notebook/experiment_2")
csv_file = base_path / "prompt_base_and_variations_full.csv"

# Đọc CSV
df = pd.read_csv(csv_file)

print(f"Tổng số dòng trong CSV: {len(df)}")
print(f"Số ảnh unique: {df['Images'].nunique()}")
print()

# Tìm ảnh bị lặp
image_counts = df['Images'].value_counts()
duplicated_images = image_counts[image_counts > 1]

print(f"Số ảnh bị lặp: {len(duplicated_images)}")
print(f"Tổng số lần lặp: {duplicated_images.sum()} (tính cả bản gốc)")
print(f"Số dòng thừa do lặp: {duplicated_images.sum() - len(duplicated_images)}")
print()

if len(duplicated_images) > 0:
    print("="*80)
    print("DANH SÁCH ẢNH BỊ LẶP:")
    print("="*80)
    
    for img_name, count in duplicated_images.items():
        print(f"\n{img_name}: xuất hiện {count} lần")
        
        # Lấy tất cả các dòng của ảnh này
        img_rows = df[df['Images'] == img_name]
        
        for idx, row in img_rows.iterrows():
            print(f"  Dòng {idx + 2}: gender={row['gender']}, image_path={row['image_path']}")
            
            # Kiểm tra xem có prompt nào không
            has_base = pd.notna(row['base_prompt'])
            has_var1 = pd.notna(row['variation_1_prompt'])
            has_var2 = pd.notna(row['variation_2_prompt'])
            has_var3 = pd.notna(row['variation_3_prompt'])
            has_var4 = pd.notna(row['variation_4_prompt'])
            
            prompts_info = []
            if has_base:
                prompts_info.append("base_prompt")
            if has_var1:
                prompts_info.append("variation_1")
            if has_var2:
                prompts_info.append("variation_2")
            if has_var3:
                prompts_info.append("variation_3")
            if has_var4:
                prompts_info.append("variation_4")
            
            if prompts_info:
                print(f"         Có prompts: {', '.join(prompts_info)}")
            else:
                print(f"         Không có prompt nào")

print("\n" + "="*80)

# Tính toán tổng hợp
total_rows = len(df)
unique_images = df['Images'].nunique()
duplicate_rows = total_rows - unique_images

print(f"\nTÓM TẮT:")
print(f"  - Tổng số dòng: {total_rows}")
print(f"  - Số ảnh unique: {unique_images}")
print(f"  - Số dòng bị lặp thừa: {duplicate_rows}")
print(f"  - Nếu xóa dòng lặp sẽ còn: {unique_images} dòng")

Tổng số dòng trong CSV: 199
Số ảnh unique: 146

Số ảnh bị lặp: 53
Tổng số lần lặp: 106 (tính cả bản gốc)
Số dòng thừa do lặp: 53

DANH SÁCH ẢNH BỊ LẶP:

1.jpg: xuất hiện 2 lần
  Dòng 2: gender=male, image_path=/home/hungnq/hungnq/sd_stuff/datahub/dataset_200/male/1.jpg
         Có prompts: base_prompt, variation_1, variation_2, variation_3, variation_4
  Dòng 101: gender=female, image_path=/home/hungnq/hungnq/sd_stuff/datahub/dataset_200/female/1.jpg
         Có prompts: base_prompt

Image_44.jpg: xuất hiện 2 lần
  Dòng 50: gender=male, image_path=/home/hungnq/hungnq/sd_stuff/datahub/dataset_200/male/Image_44.jpg
         Có prompts: base_prompt
  Dòng 155: gender=female, image_path=/home/hungnq/hungnq/sd_stuff/datahub/dataset_200/female/Image_44.jpg
         Có prompts: base_prompt

30.jpg: xuất hiện 2 lần
  Dòng 31: gender=male, image_path=/home/hungnq/hungnq/sd_stuff/datahub/dataset_200/male/30.jpg
         Có prompts: base_prompt
  Dòng 130: gender=female, image_path=/home/hungnq/h

In [6]:
# Tóm tắt ngắn gọn về ảnh bị lặp
import pandas as pd
from pathlib import Path

base_path = Path("/home/hungnq/hungnq/sd_stuff/notebook/experiment_2")
csv_file = base_path / "prompt_base_and_variations_full.csv"

# Đọc CSV
df = pd.read_csv(csv_file)

# Tìm ảnh bị lặp
image_counts = df['Images'].value_counts()
duplicated_images = image_counts[image_counts > 1]

print(f"Tổng số dòng trong CSV: {len(df)}")
print(f"Số ảnh unique: {df['Images'].nunique()}")
print(f"Số ảnh bị lặp: {len(duplicated_images)}")
print()

# Thống kê theo số lần lặp
print("THỐNG KÊ THEO SỐ LẦN LẶP:")
duplicate_count_stats = duplicated_images.value_counts().sort_index()
for count, num_images in duplicate_count_stats.items():
    print(f"  - {num_images} ảnh xuất hiện {count} lần")

print()
print("="*80)
print(f"TOP 20 ẢNH BỊ LẶP NHIỀU NHẤT:")
print("="*80)

for idx, (img_name, count) in enumerate(duplicated_images.head(20).items(), 1):
    print(f"{idx:2d}. {img_name:20s} - xuất hiện {count} lần")

print()
print("="*80)
print("TÓM TẮT:")
print("="*80)
total_rows = len(df)
unique_images = df['Images'].nunique()
duplicate_rows = total_rows - unique_images

print(f"Tổng số dòng: {total_rows}")
print(f"Số ảnh unique: {unique_images}")
print(f"Số dòng thừa do lặp: {duplicate_rows}")
print(f"Nếu giữ 1 dòng/ảnh sẽ còn: {unique_images} dòng")

# Xuất danh sách đầy đủ các ảnh bị lặp ra file
output_file = base_path / "duplicated_images_analysis.txt"
with open(output_file, 'w') as f:
    f.write("DANH SÁCH ẢNH BỊ LẶP TRONG CSV\n")
    f.write("="*80 + "\n\n")
    
    for img_name, count in duplicated_images.items():
        f.write(f"\n{img_name}: xuất hiện {count} lần\n")
        
        # Lấy tất cả các dòng của ảnh này
        img_rows = df[df['Images'] == img_name]
        
        for idx, row in img_rows.iterrows():
            f.write(f"  Dòng {idx + 2}: gender={row['gender']}\n")
            
            # Kiểm tra prompts
            prompts_info = []
            if pd.notna(row['base_prompt']):
                prompts_info.append("base_prompt")
            if pd.notna(row.get('variation_1_prompt')):
                prompts_info.append("variation_1")
            if pd.notna(row.get('variation_2_prompt')):
                prompts_info.append("variation_2")
            if pd.notna(row.get('variation_3_prompt')):
                prompts_info.append("variation_3")
            if pd.notna(row.get('variation_4_prompt')):
                prompts_info.append("variation_4")
            
            if prompts_info:
                f.write(f"         Có: {', '.join(prompts_info)}\n")

print(f"\nĐã xuất chi tiết vào file: {output_file}")

Tổng số dòng trong CSV: 199
Số ảnh unique: 146
Số ảnh bị lặp: 53

THỐNG KÊ THEO SỐ LẦN LẶP:
  - 53 ảnh xuất hiện 2 lần

TOP 20 ẢNH BỊ LẶP NHIỀU NHẤT:
 1. 1.jpg                - xuất hiện 2 lần
 2. Image_44.jpg         - xuất hiện 2 lần
 3. 30.jpg               - xuất hiện 2 lần
 4. Image_13.jpg         - xuất hiện 2 lần
 5. Image_24.jpg         - xuất hiện 2 lần
 6. Image_28.jpg         - xuất hiện 2 lần
 7. Image_3.jpg          - xuất hiện 2 lần
 8. Image_30.jpg         - xuất hiện 2 lần
 9. Image_31.jpg         - xuất hiện 2 lần
10. Image_33.jpg         - xuất hiện 2 lần
11. Image_34.jpg         - xuất hiện 2 lần
12. Image_39.jpg         - xuất hiện 2 lần
13. Image_4.jpg          - xuất hiện 2 lần
14. Image_48.jpg         - xuất hiện 2 lần
15. 28.jpg               - xuất hiện 2 lần
16. Image_49.jpg         - xuất hiện 2 lần
17. Image_51.jpg         - xuất hiện 2 lần
18. Image_56.jpg         - xuất hiện 2 lần
19. Image_62.jpg         - xuất hiện 2 lần
20. Image_65.jpg         - xuất h

In [7]:
# Điều tra tại sao prompt_variations_images_batch_0_50_2 và batch_100_150 chỉ có ~30-40 ảnh
import json
from pathlib import Path
import os

base_path = Path("/home/hungnq/hungnq/sd_stuff/notebook/experiment_2")

# Danh sách các folders cần kiểm tra
folders_to_check = [
    "prompt_variations_images_batch_0_50_2",
    "prompt_variations_images_batch_100_150"
]

print("="*80)
print("PHÂN TÍCH CHI TIẾT 2 FOLDERS CÓ ÍT ẢNH")
print("="*80)

for folder_name in folders_to_check:
    folder_path = base_path / folder_name
    metadata_path = folder_path / "metadata.json"
    
    print(f"\n📁 {folder_name}")
    print("-" * 80)
    
    # Đọc metadata
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    
    print(f"Số entries trong metadata.json: {len(metadata)}")
    
    # Lấy danh sách image folders
    image_folders = [d for d in os.listdir(folder_path) 
                    if d.startswith('image_') and os.path.isdir(folder_path / d)]
    
    print(f"Số image folders: {len(image_folders)}")
    
    # Đếm các loại ảnh
    image_types = {}
    total_images = 0
    empty_folders = []
    
    for img_folder in sorted(image_folders):
        img_folder_path = folder_path / img_folder
        images = list(img_folder_path.glob('*.png')) + list(img_folder_path.glob('*.jpg'))
        
        if len(images) == 0:
            empty_folders.append(img_folder)
        else:
            total_images += len(images)
            for img in images:
                stem = img.stem  # base_male, base_female, etc.
                image_types[stem] = image_types.get(stem, 0) + 1
    
    print(f"Tổng số ảnh thực tế: {total_images}")
    print(f"Folders trống: {len(empty_folders)}")
    
    if empty_folders:
        print(f"  Danh sách folders trống: {empty_folders}")
    
    print(f"\nPhân loại ảnh theo loại:")
    for img_type, count in sorted(image_types.items()):
        print(f"  - {img_type}: {count} ảnh")
    
    # Phân tích metadata để xem nên có bao nhiêu ảnh
    print(f"\nPhân tích metadata:")
    metadata_by_type = {}
    for key, value in metadata.items():
        prompt_type = value.get('prompt_type', 'unknown')
        metadata_by_type[prompt_type] = metadata_by_type.get(prompt_type, 0) + 1
    
    print(f"Metadata entries theo prompt_type:")
    for ptype, count in sorted(metadata_by_type.items()):
        print(f"  - {ptype}: {count} entries")
    
    # Kiểm tra image_id trong metadata
    image_ids = set()
    for key, value in metadata.items():
        if 'image_id' in value:
            image_ids.add(value['image_id'])
    
    print(f"\nSố image_id unique trong metadata: {len(image_ids)}")
    print(f"Image IDs range: {min(image_ids) if image_ids else 'N/A'} - {max(image_ids) if image_ids else 'N/A'}")
    
    # Kiểm tra xem có image_id nào thiếu không
    if image_ids:
        expected_range = set(range(min(image_ids), max(image_ids) + 1))
        missing_ids = expected_range - image_ids
        if missing_ids:
            print(f"Image IDs bị thiếu ({len(missing_ids)}): {sorted(missing_ids)[:20]}")

print("\n" + "="*80)

PHÂN TÍCH CHI TIẾT 2 FOLDERS CÓ ÍT ẢNH

📁 prompt_variations_images_batch_0_50_2
--------------------------------------------------------------------------------
Số entries trong metadata.json: 38
Số image folders: 38
Tổng số ảnh thực tế: 38
Folders trống: 0

Phân loại ảnh theo loại:
  - base_male: 38 ảnh

Phân tích metadata:
Metadata entries theo prompt_type:
  - base: 38 entries

Số image_id unique trong metadata: 38
Image IDs range: 1 - 47
Image IDs bị thiếu (9): [32, 35, 37, 40, 41, 42, 43, 45, 46]

📁 prompt_variations_images_batch_100_150
--------------------------------------------------------------------------------
Số entries trong metadata.json: 36
Số image folders: 37
Tổng số ảnh thực tế: 36
Folders trống: 1
  Danh sách folders trống: ['image_0037']

Phân loại ảnh theo loại:
  - base_female: 36 ảnh

Phân tích metadata:
Metadata entries theo prompt_type:
  - base: 36 entries

Số image_id unique trong metadata: 36
Image IDs range: 2 - 42
Image IDs bị thiếu (5): [35, 36, 37, 39, 

In [1]:
# Điều tra cột image_path trong CSV
import pandas as pd
from pathlib import Path
import os

base_path = Path("/home/hungnq/hungnq/sd_stuff/notebook/experiment_2")
csv_file = base_path / "prompt_base_and_variations_full.csv"

# Đọc CSV
df = pd.read_csv(csv_file)

print("="*80)
print("PHÂN TÍCH CỘT image_path TRONG CSV")
print("="*80)

print(f"\nTổng số dòng trong CSV: {len(df)}")

# Kiểm tra cột image_path có null/empty không
null_image_paths = df['image_path'].isna().sum()
empty_image_paths = (df['image_path'] == '').sum()

print(f"\nDòng có image_path null (NaN): {null_image_paths}")
print(f"Dòng có image_path empty (''): {empty_image_paths}")
print(f"Dòng có image_path hợp lệ: {len(df) - null_image_paths - empty_image_paths}")

# Kiểm tra xem các đường dẫn có tồn tại không
print("\n" + "="*80)
print("KIỂM TRA SỰ TỒN TẠI CỦA FILES")
print("="*80)

existing_files = 0
missing_files = 0
missing_file_list = []

for idx, row in df.iterrows():
    image_path = row['image_path']
    
    if pd.notna(image_path) and image_path != '':
        if os.path.exists(image_path):
            existing_files += 1
        else:
            missing_files += 1
            missing_file_list.append({
                'row': idx + 2,  # +2 vì CSV có header và index bắt đầu từ 0
                'image': row['Images'],
                'gender': row['gender'],
                'path': image_path
            })

print(f"\nFiles tồn tại: {existing_files}")
print(f"Files KHÔNG tồn tại: {missing_files}")

if missing_files > 0:
    print(f"\n{'='*80}")
    print(f"DANH SÁCH {min(20, len(missing_file_list))} FILES ĐẦU TIÊN KHÔNG TỒN TẠI:")
    print(f"{'='*80}")
    
    for item in missing_file_list[:20]:
        print(f"Dòng {item['row']}: {item['image']} ({item['gender']})")
        print(f"  Path: {item['path']}")

# Phân tích cấu trúc đường dẫn
print("\n" + "="*80)
print("PHÂN TÍCH CẤU TRÚC ĐƯỜNG DẪN")
print("="*80)

# Lấy các đường dẫn unique
unique_paths = df['image_path'].dropna().unique()
print(f"\nSố đường dẫn unique: {len(unique_paths)}")

# Phân loại theo thư mục
path_prefixes = {}
for path in unique_paths:
    if path:
        # Lấy thư mục cha
        parent_dir = os.path.dirname(path)
        path_prefixes[parent_dir] = path_prefixes.get(parent_dir, 0) + 1

print(f"\nPhân loại theo thư mục:")
for prefix, count in sorted(path_prefixes.items(), key=lambda x: x[1], reverse=True):
    print(f"  {prefix}: {count} files")

PHÂN TÍCH CỘT image_path TRONG CSV

Tổng số dòng trong CSV: 199

Dòng có image_path null (NaN): 0
Dòng có image_path empty (''): 0
Dòng có image_path hợp lệ: 199

KIỂM TRA SỰ TỒN TẠI CỦA FILES

Files tồn tại: 199
Files KHÔNG tồn tại: 0

PHÂN TÍCH CẤU TRÚC ĐƯỜNG DẪN

Số đường dẫn unique: 199

Phân loại theo thư mục:
  /home/hungnq/hungnq/sd_stuff/datahub/dataset_200/female: 100 files
  /home/hungnq/hungnq/sd_stuff/datahub/dataset_200/male: 99 files
